In [ ]:
# Interactive Video/Audio Processing Toolkit for Google Colab
# Install required packages first - run this cell first

!apt update &> /dev/null
!apt install ffmpeg &> /dev/null
!pip install -q openai-whisper moviepy librosa noisereduce pydub

import os
import re
import subprocess
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import librosa
import noisereduce as nr
import numpy as np
from pydub import AudioSegment
import whisper
from moviepy.editor import VideoFileClip, AudioFileClip
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, Audio, Video, clear_output

class VideoAudioProcessor:
    """Interactive Video/Audio Processing System"""

    def __init__(self):
        self.uploaded_files = []
        self.whisper_model = None
        self.current_file = None

    def start(self):
        """Start the interactive processing system"""
        clear_output()
        print("🎬" + "="*60 + "🎵")
        print("    INTERACTIVE VIDEO/AUDIO PROCESSING TOOLKIT")
        print("="*64)
        print()

        while True:
            self.show_main_menu()
            try:
                choice = input("\n👉 Enter your choice (1-6): ").strip()

                if choice == '1':
                    self.handle_file_upload()
                elif choice == '2':
                    self.handle_format_conversion()
                elif choice == '3':
                    self.handle_trimming_cropping()
                elif choice == '4':
                    self.handle_ai_processing()
                elif choice == '5':
                    self.handle_natural_language()
                elif choice == '6':
                    print("\n👋 Thank you for using the toolkit! Goodbye!")
                    break
                else:
                    print("\n❌ Invalid choice! Please enter 1-6.")

                self.wait_for_user()

            except KeyboardInterrupt:
                print("\n\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ An error occurred: {str(e)}")
                self.wait_for_user()

    def show_main_menu(self):
        """Display the main menu"""
        clear_output()
        print("🎬" + "="*60 + "🎵")
        print("    INTERACTIVE VIDEO/AUDIO PROCESSING TOOLKIT")
        print("="*64)

        if self.uploaded_files:
            print(f"📁 Uploaded files: {', '.join(self.uploaded_files[:3])}" +
                  (f" (+{len(self.uploaded_files)-3} more)" if len(self.uploaded_files) > 3 else ""))
        else:
            print("📁 No files uploaded yet")

        print("\n🔧 AVAILABLE TASKS:")
        print("1️⃣  Upload Files")
        print("2️⃣  Format Conversion & Compression")
        print("3️⃣  Trimming & Cropping")
        print("4️⃣  AI Processing (Noise Removal, Subtitles)")
        print("5️⃣  Natural Language Commands")
        print("6️⃣  Exit")

    def handle_file_upload(self):
        """Handle file upload process"""
        clear_output()
        print("📁 FILE UPLOAD")
        print("="*40)

        print("\n🔄 Please select files to upload...")
        uploaded = files.upload()

        if uploaded:
            self.uploaded_files.extend(uploaded.keys())
            print(f"\n✅ Successfully uploaded {len(uploaded)} file(s):")
            for filename in uploaded.keys():
                file_size = len(uploaded[filename]) / 1024 / 1024  # MB
                print(f"   📄 {filename} ({file_size:.1f} MB)")
        else:
            print("\n❌ No files were uploaded.")

    def handle_format_conversion(self):
        """Handle format conversion and compression"""
        clear_output()
        print("🔄 FORMAT CONVERSION & COMPRESSION")
        print("="*40)

        if not self.uploaded_files:
            print("\n❌ No files uploaded! Please upload files first.")
            return

        # Show available files
        print("\n📁 Available files:")
        for i, filename in enumerate(self.uploaded_files, 1):
            print(f"   {i}. {filename}")

        # Get file selection
        try:
            file_idx = int(input(f"\n👉 Select file (1-{len(self.uploaded_files)}): ")) - 1
            input_file = self.uploaded_files[file_idx]
        except (ValueError, IndexError):
            print("❌ Invalid file selection!")
            return

        print(f"\n📄 Selected: {input_file}")

        # Show conversion options
        print("\n🔄 CONVERSION OPTIONS:")
        print("1️⃣  MP4 to MP3 (High Quality)")
        print("2️⃣  MP4 to MP3 (Medium Quality)")
        print("3️⃣  MP4 to MP3 (Low Quality)")
        print("4️⃣  Video Compression (Target Size)")
        print("5️⃣  Video Compression (Quality Based)")
        print("6️⃣  Custom Format Conversion")

        choice = input("\n👉 Select conversion type (1-6): ").strip()

        if choice in ['1', '2', '3']:
            quality_map = {'1': 'high', '2': 'medium', '3': 'low'}
            quality = quality_map[choice]

            output_file = input(f"\n📝 Output filename (press Enter for auto): ").strip()
            if not output_file:
                output_file = Path(input_file).stem + f'_{quality}.mp3'

            print(f"\n🔄 Converting {input_file} to MP3 ({quality} quality)...")
            result = self.convert_mp4_to_mp3(input_file, output_file, quality)
            print(result)

        elif choice == '4':
            target_mb = input("\n📏 Target size in MB (e.g., 50): ").strip()
            try:
                target_mb = int(target_mb)
                output_file = input(f"\n📝 Output filename (press Enter for auto): ").strip()
                if not output_file:
                    output_file = Path(input_file).stem + f'_compressed_{target_mb}mb.mp4'

                print(f"\n🔄 Compressing video to {target_mb}MB...")
                result = self.compress_video(input_file, output_file, target_mb)
                print(result)
            except ValueError:
                print("❌ Invalid size! Please enter a number.")

        elif choice == '5':
            print("\n🎚️ Quality levels:")
            print("   1. High (CRF 18) - Large file, best quality")
            print("   2. Medium (CRF 23) - Balanced")
            print("   3. Low (CRF 28) - Small file, lower quality")

            quality_choice = input("\n👉 Select quality (1-3): ").strip()
            crf_map = {'1': 18, '2': 23, '3': 28}

            if quality_choice in crf_map:
                crf = crf_map[quality_choice]
                output_file = input(f"\n📝 Output filename (press Enter for auto): ").strip()
                if not output_file:
                    output_file = Path(input_file).stem + f'_compressed_crf{crf}.mp4'

                print(f"\n🔄 Compressing video with CRF {crf}...")
                result = self.compress_video_crf(input_file, output_file, crf)
                print(result)
            else:
                print("❌ Invalid quality selection!")

        elif choice == '6':
            target_format = input("\n📝 Target format (e.g., avi, mov, wav): ").strip().lower()
            output_file = input(f"\n📝 Output filename (press Enter for auto): ").strip()
            if not output_file:
                output_file = Path(input_file).stem + f'.{target_format}'

            print(f"\n🔄 Converting to {target_format.upper()}...")
            result = self.convert_format(input_file, output_file)
            print(result)

    def handle_trimming_cropping(self):
        """Handle trimming and cropping operations"""
        clear_output()
        print("✂️ TRIMMING & CROPPING")
        print("="*40)

        if not self.uploaded_files:
            print("\n❌ No files uploaded! Please upload files first.")
            return

        # Show available files
        print("\n📁 Available files:")
        for i, filename in enumerate(self.uploaded_files, 1):
            print(f"   {i}. {filename}")

        # Get file selection
        try:
            file_idx = int(input(f"\n👉 Select file (1-{len(self.uploaded_files)}): ")) - 1
            input_file = self.uploaded_files[file_idx]
        except (ValueError, IndexError):
            print("❌ Invalid file selection!")
            return

        print(f"\n📄 Selected: {input_file}")

        # Show trimming options
        print("\n✂️ TRIMMING/CROPPING OPTIONS:")
        print("1️⃣  Trim Video/Audio (Start to End time)")
        print("2️⃣  Trim Video/Audio (Start + Duration)")
        print("3️⃣  Crop Video (Width x Height)")
        print("4️⃣  Crop Video (Width x Height at X,Y position)")

        choice = input("\n👉 Select operation (1-4): ").strip()

        if choice == '1':
            start_time = input("\n⏰ Start time (HH:MM:SS or MM:SS): ").strip()
            end_time = input("⏰ End time (HH:MM:SS or MM:SS): ").strip()
            output_file = input("📝 Output filename (press Enter for auto): ").strip()

            if not output_file:
                output_file = Path(input_file).stem + '_trimmed' + Path(input_file).suffix

            print(f"\n✂️ Trimming from {start_time} to {end_time}...")
            result = self.trim_media(input_file, start_time, end_time, None, output_file)
            print(result)

        elif choice == '2':
            start_time = input("\n⏰ Start time (HH:MM:SS or MM:SS): ").strip()
            duration = input("⏱️ Duration (HH:MM:SS or seconds): ").strip()
            output_file = input("📝 Output filename (press Enter for auto): ").strip()

            if not output_file:
                output_file = Path(input_file).stem + '_trimmed' + Path(input_file).suffix

            print(f"\n✂️ Trimming {duration} from {start_time}...")
            result = self.trim_media(input_file, start_time, None, duration, output_file)
            print(result)

        elif choice in ['3', '4']:
            try:
                width = int(input("\n📐 Width: "))
                height = int(input("📐 Height: "))

                if choice == '4':
                    x = int(input("📍 X position: "))
                    y = int(input("📍 Y position: "))
                else:
                    x, y = 0, 0

                output_file = input("📝 Output filename (press Enter for auto): ").strip()
                if not output_file:
                    output_file = Path(input_file).stem + '_cropped.mp4'

                print(f"\n✂️ Cropping to {width}x{height} at ({x},{y})...")
                result = self.crop_video(input_file, width, height, x, y, output_file)
                print(result)

            except ValueError:
                print("❌ Invalid dimensions! Please enter numbers only.")

    def handle_ai_processing(self):
        """Handle AI processing operations"""
        clear_output()
        print("🤖 AI PROCESSING")
        print("="*40)

        if not self.uploaded_files:
            print("\n❌ No files uploaded! Please upload files first.")
            return

        # Show available files
        print("\n📁 Available files:")
        for i, filename in enumerate(self.uploaded_files, 1):
            print(f"   {i}. {filename}")

        # Get file selection
        try:
            file_idx = int(input(f"\n👉 Select file (1-{len(self.uploaded_files)}): ")) - 1
            input_file = self.uploaded_files[file_idx]
        except (ValueError, IndexError):
            print("❌ Invalid file selection!")
            return

        print(f"\n📄 Selected: {input_file}")

        # Show AI processing options
        print("\n🤖 AI PROCESSING OPTIONS:")
        print("1️⃣  Audio Noise Removal")
        print("2️⃣  Generate Subtitles")
        print("3️⃣  Smart Video Compression")
        print("4️⃣  Extract Audio from Video")

        choice = input("\n👉 Select AI operation (1-4): ").strip()

        if choice == '1':
            output_file = input("\n📝 Output filename (press Enter for auto): ").strip()
            if not output_file:
                output_file = Path(input_file).stem + '_denoised.wav'

            print("\n🔄 Removing noise using AI algorithms...")
            print("⏳ This may take a few moments...")
            result = self.remove_noise(input_file, output_file)
            print(result)

        elif choice == '2':
            print("\n🌍 Available languages:")
            print("   • auto (Auto-detect)")
            print("   • en (English)")
            print("   • es (Spanish)")
            print("   • fr (French)")
            print("   • de (German)")
            print("   • it (Italian)")
            print("   • ja (Japanese)")
            print("   • ko (Korean)")

            language = input("\n👉 Language code (or 'auto'): ").strip().lower()
            if not language:
                language = 'auto'

            output_file = input("📝 SRT filename (press Enter for auto): ").strip()
            if not output_file:
                output_file = Path(input_file).stem + '.srt'

            print(f"\n🔄 Generating subtitles in {language}...")
            print("⏳ Loading AI model, this may take a few moments...")
            result = self.generate_subtitles(input_file, output_file, language)
            print(result)

        elif choice == '3':
            print("\n🎚️ Smart compression quality:")
            print("   1. High - Best quality, larger file")
            print("   2. Balanced - Good quality, moderate size")
            print("   3. Fast - Lower quality, smallest file")

            quality_choice = input("\n👉 Select quality (1-3): ").strip()
            quality_map = {'1': 'high', '2': 'balanced', '3': 'fast'}

            if quality_choice in quality_map:
                quality = quality_map[quality_choice]
                output_file = input("📝 Output filename (press Enter for auto): ").strip()
                if not output_file:
                    output_file = Path(input_file).stem + f'_smart_{quality}.mp4'

                print(f"\n🔄 Smart compression with {quality} quality...")
                result = self.smart_compress(input_file, output_file, quality)
                print(result)
            else:
                print("❌ Invalid quality selection!")

        elif choice == '4':
            output_file = input("\n📝 Audio filename (press Enter for auto): ").strip()
            if not output_file:
                output_file = Path(input_file).stem + '_audio.wav'

            print("\n🔄 Extracting audio from video...")
            result = self.extract_audio(input_file, output_file)
            print(result)

    def handle_natural_language(self):
        """Handle natural language commands"""
        clear_output()
        print("💬 NATURAL LANGUAGE COMMANDS")
        print("="*40)

        if not self.uploaded_files:
            print("\n❌ No files uploaded! Please upload files first.")
            return

        print("\n📁 Available files:")
        for i, filename in enumerate(self.uploaded_files, 1):
            print(f"   {i}. {filename}")

        try:
            file_idx = int(input(f"\n👉 Select file (1-{len(self.uploaded_files)}): ")) - 1
            input_file = self.uploaded_files[file_idx]
        except (ValueError, IndexError):
            print("❌ Invalid file selection!")
            return

        print(f"\n📄 Selected: {input_file}")

        print("\n💡 EXAMPLE COMMANDS:")
        print("   • 'convert to mp3'")
        print("   • 'compress to 50MB'")
        print("   • 'trim from 00:30 to 01:30'")
        print("   • 'extract 60 seconds starting at 01:00'")
        print("   • 'resize to 720x480'")
        print("   • 'crop to 500x500'")

        command = input("\n💬 Enter your command: ").strip()

        if command:
            print(f"\n🔄 Processing: '{command}'...")
            result = self.process_natural_language(command, input_file)
            print(result)
        else:
            print("❌ No command entered!")

    def wait_for_user(self):
        """Wait for user to press Enter before continuing"""
        input("\n⏎ Press Enter to continue...")

    # Processing Functions
    def convert_mp4_to_mp3(self, input_file: str, output_file: str, quality: str = 'high') -> str:
        """Convert MP4 to MP3"""
        quality_settings = {
            'high': '-b:a 320k',
            'medium': '-b:a 192k',
            'low': '-b:a 128k'
        }

        command = f'ffmpeg -i "{input_file}" {quality_settings[quality]} "{output_file}" -y'
        return self._run_ffmpeg(command)

    def compress_video(self, input_file: str, output_file: str, target_mb: int) -> str:
        """Compress video to target size"""
        # Get duration for bitrate calculation
        duration = self._get_duration(input_file)
        if duration == 0:
            return "❌ Could not determine video duration"

        # Calculate target bitrate (with some buffer)
        target_bitrate = int((target_mb * 8 * 1024) / duration * 0.95)  # 95% to leave room for audio

        command = f'ffmpeg -i "{input_file}" -b:v {target_bitrate}k -maxrate {int(target_bitrate*1.2)}k -bufsize {int(target_bitrate*2)}k "{output_file}" -y'
        return self._run_ffmpeg(command)

    def compress_video_crf(self, input_file: str, output_file: str, crf: int) -> str:
        """Compress video using CRF"""
        command = f'ffmpeg -i "{input_file}" -crf {crf} "{output_file}" -y'
        return self._run_ffmpeg(command)

    def convert_format(self, input_file: str, output_file: str) -> str:
        """Convert to different format"""
        command = f'ffmpeg -i "{input_file}" "{output_file}" -y'
        return self._run_ffmpeg(command)

    def trim_media(self, input_file: str, start_time: str, end_time: str = None, duration: str = None, output_file: str = None) -> str:
        """Trim video or audio"""
        if end_time:
            command = f'ffmpeg -i "{input_file}" -ss {start_time} -to {end_time} -c copy "{output_file}" -y'
        elif duration:
            command = f'ffmpeg -i "{input_file}" -ss {start_time} -t {duration} -c copy "{output_file}" -y'
        else:
            command = f'ffmpeg -i "{input_file}" -ss {start_time} -c copy "{output_file}" -y'

        return self._run_ffmpeg(command)

    def crop_video(self, input_file: str, width: int, height: int, x: int = 0, y: int = 0, output_file: str = None) -> str:
        """Crop video"""
        command = f'ffmpeg -i "{input_file}" -vf crop={width}:{height}:{x}:{y} "{output_file}" -y'
        return self._run_ffmpeg(command)

    def remove_noise(self, input_file: str, output_file: str) -> str:
        """Remove noise from audio"""
        try:
            audio_data, sample_rate = librosa.load(input_file, sr=None)
            denoised = nr.reduce_noise(y=audio_data, sr=sample_rate, stationary=False, prop_decrease=0.8)

            # Use scipy to save (librosa.output.write_wav is deprecated)
            import scipy.io.wavfile as wav
            wav.write(output_file, sample_rate, (denoised * 32767).astype(np.int16))

            return f"✅ Noise removal completed! Saved to: {output_file}"
        except Exception as e:
            return f"❌ Error in noise removal: {str(e)}"

    def generate_subtitles(self, input_file: str, output_file: str, language: str = 'auto') -> str:
        """Generate subtitles using Whisper"""
        try:
            if self.whisper_model is None:
                self.whisper_model = whisper.load_model("base")

            result = self.whisper_model.transcribe(
                input_file,
                language=None if language == 'auto' else language
            )

            # Convert to SRT
            srt_content = self._convert_to_srt(result['segments'])

            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(srt_content)

            return f"✅ Subtitles generated! Saved to: {output_file}"
        except Exception as e:
            return f"❌ Error generating subtitles: {str(e)}"

    def smart_compress(self, input_file: str, output_file: str, quality: str) -> str:
        """Smart video compression"""
        quality_settings = {
            'high': {'crf': 18, 'preset': 'slow'},
            'balanced': {'crf': 23, 'preset': 'medium'},
            'fast': {'crf': 28, 'preset': 'fast'}
        }

        settings = quality_settings[quality]
        command = f'ffmpeg -i "{input_file}" -crf {settings["crf"]} -preset {settings["preset"]} "{output_file}" -y'
        return self._run_ffmpeg(command)

    def extract_audio(self, input_file: str, output_file: str) -> str:
        """Extract audio from video"""
        command = f'ffmpeg -i "{input_file}" -vn -acodec pcm_s16le "{output_file}" -y'
        return self._run_ffmpeg(command)

    def process_natural_language(self, command: str, input_file: str) -> str:
        """Process natural language command"""
        command = command.lower().strip()
        output_file = Path(input_file).stem + "_processed"

        if "convert" in command and "mp3" in command:
            return self.convert_mp4_to_mp3(input_file, output_file + ".mp3")
        elif "compress" in command:
            if "mb" in command:
                # Extract number
                import re
                match = re.search(r'(\d+)\s*mb', command)
                if match:
                    target_mb = int(match.group(1))
                    return self.compress_video(input_file, output_file + ".mp4", target_mb)
            return self.compress_video_crf(input_file, output_file + ".mp4", 23)
        elif "trim" in command:
            # Extract times
            time_pattern = r'(\d{1,2}:\d{2}(?::\d{2})?)'
            times = re.findall(time_pattern, command)
            if len(times) >= 2:
                return self.trim_media(input_file, times[0], times[1], None, output_file + Path(input_file).suffix)
        elif "extract" in command and "audio" in command:
            return self.extract_audio(input_file, output_file + ".wav")

        return f"❌ Could not understand command: '{command}'"

    def _run_ffmpeg(self, command: str) -> str:
        """Run ffmpeg command"""
        print(f"🔧 Running: {command}")
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        if result.returncode == 0:
            return "✅ Operation completed successfully!"
        else:
            return f"❌ Error: {result.stderr}"

    def _get_duration(self, input_file: str) -> float:
        """Get media duration"""
        cmd = f'ffprobe -v quiet -show_entries format=duration -of csv=p=0 "{input_file}"'
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        try:
            return float(result.stdout.strip())
        except:
            return 0

    def _convert_to_srt(self, segments: List[Dict]) -> str:
        """Convert Whisper segments to SRT"""
        srt_content = ""
        for i, segment in enumerate(segments):
            start_time = self._seconds_to_srt_time(segment['start'])
            end_time = self._seconds_to_srt_time(segment['end'])
            text = segment['text'].strip()

            srt_content += f"{i+1}\n{start_time} --> {end_time}\n{text}\n\n"

        return srt_content

    def _seconds_to_srt_time(self, seconds: float) -> str:
        """Convert seconds to SRT time format"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        milliseconds = int((seconds % 1) * 1000)

        return f"{hours:02d}:{minutes:02d}:{secs:02d},{milliseconds:03d}"

# Initialize and start the interactive processor
print("🚀 Initializing Interactive Video/Audio Processing Toolkit...")
print("📦 All dependencies installed successfully!")
print()

processor = VideoAudioProcessor()
processor.start()